## test

In [ ]:
import os

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

## import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, Flatten
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import backend as K
from livelossplot.inputs.tf_keras import PlotLossesCallback

## lable preparation

In [ ]:
tr_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/train_faces.csv')
ts_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/test_faces.csv')
vl_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/val_faces.csv')

## functions

In [ ]:
def soft_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size):
        self.n = df['name'].tolist()
        self.df = df
        self.batch_size = batch_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.n) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp = [self.n[k] for k in indexes]
        x, y = self.__data_generation(list_IDs_temp, indexes)
        return x, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.n))

    def __data_generation(self, list_IDs_temp, indexes):
        images = np.empty((self.batch_size, 384, 384, 1))
        face = np.empty((self.batch_size, 4))

        for i, ID in enumerate(list_IDs_temp):
            train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ID, color_mode = "grayscale", target_size = (
            384, 512)))
            train_img = train_img[:, 64:448]
            train_img /= 255.0
            images[i,] = train_img

            face[i, 0] = int(self.df.iloc[indexes[i]]['Xmax']) // 2
            face[i, 1] = int(self.df.iloc[indexes[i]]['Xmin']) // 2
            face[i, 2] = int(self.df.iloc[indexes[i]]['Ymin']) // 2
            face[i, 3] = int(self.df.iloc[indexes[i]]['Ymax']) // 2

        return images, face


bs = 8
train_gen = DataGenerator(tr_df, bs)
val_gen = DataGenerator(vl_df, bs)
test_gen = DataGenerator(ts_df, bs)

## model structure

In [ ]:
def PSNR(super_resolution, high_resolution):
    psnr_value = tf.image.psnr(high_resolution, super_resolution, max_val = 1)[0]
    return psnr_value


def SSIMLoss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))


def vgg_block(layer_in, n_filters, n_conv):
    for _ in range(n_conv):
        layer_in = Conv2D(n_filters, (3, 3), padding = 'same', activation = 'relu')(layer_in)
    layer_in = BatchNormalization()(layer_in)
    layer_in = AveragePooling2D((2, 2), strides = (2, 2))(layer_in)
    return layer_in


def get_model():

    ######################################  INITIALIZATION  ########################################

    metrics = [tf.keras.metrics.MeanAbsolutePercentageError()]
    opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
    loss = tf.keras.losses.MeanSquaredError()
    drp = 0.0
    act = 'relu'

    ########################################  BACKBONE  ############################################
    img_input = Input(shape = (384, 384, 1))

    layer = vgg_block(img_input, 8, 2)
    layer = vgg_block(layer, 16, 2)
    layer = vgg_block(layer, 32, 2)
    layer = vgg_block(layer, 64, 2)
    layer = vgg_block(layer, 128, 2)
    layer = vgg_block(layer, 256, 2)

    ########################################  FLATTEN  #############################################

    x = Flatten()(layer)

    x = Dense(128, kernel_initializer = 'normal', activation = act)(x)
    x = Dropout(drp)(x)

    last = Dense(4)(x)

    modell = Model(inputs = img_input, outputs = last)
    modell.compile(loss = loss, optimizer = opt, metrics = metrics)

    return modell


model = get_model()
model.summary()

## train

In [ ]:
def tr():
    chk = ModelCheckpoint(filepath = 'supervised_checkpoint_face', monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
    ers = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 75)
    rduce_lr = ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 5, verbose = 1, mode = "min", min_lr = 0.000001)
    vk = PlotLossesCallback()
    return [chk, vk, rduce_lr]


cll = tr()
history = model.fit(train_gen,
                    validation_data = val_gen,
                    batch_size = bs,
                    epochs = 100,
                    verbose = 1,
                    callbacks = cll,
                    steps_per_epoch = len(tr_df) // bs,
                    validation_steps = len(vl_df) // bs
                    )

## evaluation

In [ ]:
img = np.zeros((1, 384, 384, 1)).astype('double')
n = ts_df['name'].tolist()
i = 2

train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + n[i], color_mode = "grayscale", target_size = (
384, 512))
train_img = image.img_to_array(train_igg)
train_img = train_img[:, 64:448]
train_img /= 255.0
img[0, :, :, :] = train_img
img = np.array(img)

classifier = load_model('supervised_checkpoint_face')
lbl_c = classifier.predict(img)

# summ = 0
# for i in range(100):
#     start = time()
#     lbl_c = classifier.predict(img)
#     end = time()
#     summ = summ + (end-start)

a = np.ones(np.shape(train_igg)[0])
b = np.ones(np.shape(train_igg)[1])

plt.figure(figsize = (20, 10))
plt.imshow(train_igg)

plt.plot(lbl_c[0][2] * b)
plt.plot(lbl_c[0][3] * b)
plt.plot((lbl_c[0][1] + 64) * a, range(np.shape(train_igg)[0]))
plt.plot((lbl_c[0][0] + 64) * a, range(np.shape(train_igg)[0]))
plt.show()

## test

In [ ]:
tst_loss, tst_acc = classifier.evaluate(test_gen, steps = len(ts_df) // bs)

In [ ]:
def image_generator(ts_df):
    n = ts_df.name.to_list()
    images = np.empty((len(n), 384, 384, 1))
    iou = np.zeros((len(n), 384, 384, 1), dtype = int)
    face = np.empty((len(n), 4))

    for i, ID in enumerate(n):
        train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ID, color_mode = "grayscale", target_size = (
        384, 512)))
        train_img = train_img[:, 64:448]
        train_img /= 255.0
        images[i,] = train_img

        face[i, 0] = int(ts_df.iloc[i]['Xmax']) // 2
        face[i, 1] = int(ts_df.iloc[i]['Xmin']) // 2
        face[i, 2] = int(ts_df.iloc[i]['Ymin']) // 2
        face[i, 3] = int(ts_df.iloc[i]['Ymax']) // 2

        iou[i, int(face[i, 2]):int(face[i, 3]), int(face[i, 1]):int(face[i, 0]), 0] = 1

    return images, iou


x_test, y_true = image_generator(ts_df)

In [ ]:
predictions = classifier.predict(x_test, batch_size = 1, verbose = 1, steps = len(ts_df) // 1)

In [ ]:
iou_pred = np.zeros((len(predictions), 384, 384, 1), dtype = int)

for idx, pred in enumerate(predictions):
    iou_pred[idx, int(pred[2]):int(pred[3]), int(pred[1]):int(pred[0]), 0] = 1

In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes = 2)
m.update_state(y_true, iou_pred)
mean_iou = m.result().numpy()

print('mean iou: ', mean_iou)

In [ ]:
each_iou = np.zeros((len(predictions)))

for i in range(len(predictions)):
    mymetric = tf.keras.metrics.MeanIoU(num_classes = 2)
    mymetric.update_state(y_true[i], iou_pred[i])
    each_iou[i] = mymetric.result().numpy()

In [ ]:
r1 = np.mean(each_iou)
print("\nMean: ", r1)

r2 = np.std(each_iou)
print("\nstd: ", r2)